In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.chrome.options import Options

import pandas as pd
import numpy as np

import time

from haversine import haversine

In [11]:
dir = 'Chile España 485, Ñuñoa, Chile'

In [12]:
def get_driver():
    chrome_options = Options()
    #chrome_options.add_argument("--headless")
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    chrome_options.add_experimental_option("prefs",prefs)
    s = ChromeService("chromedriver")
    driver = webdriver.Chrome(service=s, options=chrome_options)
    driver.maximize_window()
    return driver

In [13]:
def obtener_coordenadas(dir:str):

    driver = get_driver()
    
    driver.get("https://www.google.com/maps/search/"+dir)

    time.sleep(5)

    try:
        wait(driver, 4).until(ec.visibility_of_element_located((By.XPATH, "html/body/div[@id='app-container']/div[@id='content-container']/div[@id='QA0Szd']/div/div[@class='XltNde tTVLSc']/div[@class='w6VYqd']/div[@class='bJzME tTVLSc']/div[@class='k7jAl lJ3Kh miFGmb']/div[@tabindex='-1']/div[@class='aIFcqe']/div[@class='m6QErb WNBkOb ']/div[@class='TIHn2 ']/div[@class='tAiQdd']/div[@class='lMbq3e']")))

        coordenadas = driver.find_element(By.CSS_SELECTOR, 'meta[itemprop=image]').get_attribute('content')

        latitud = float(coordenadas[coordenadas.find('-'):coordenadas.find("%")])
        longitud =float(coordenadas[coordenadas.find('-', coordenadas.find("-")+1):coordenadas.find("&")])

        wait(driver, 4).until(ec.visibility_of_element_located((By.XPATH, "html/body/div[@id='app-container']/div[@id='content-container']/div[@id='QA0Szd']/div/div[@class='XltNde tTVLSc']/div[@class='w6VYqd']/div[@class='bJzME tTVLSc']/div[@class='k7jAl lJ3Kh miFGmb']/div[@tabindex='-1']/div[@class='aIFcqe']/div[@class='m6QErb WNBkOb ']/div[@class='ZKCDEc']/div[@class='RZ66Rb FgCUCc']/button[@class='aoRNLd kn2E5e NMjTrf lvtCsd ']/img")))
        imagen = driver.find_element(By.XPATH, "html/body/div[@id='app-container']/div[@id='content-container']/div[@id='QA0Szd']/div/div[@class='XltNde tTVLSc']/div[@class='w6VYqd']/div[@class='bJzME tTVLSc']/div[@class='k7jAl lJ3Kh miFGmb']/div[@tabindex='-1']/div[@class='aIFcqe']/div[@class='m6QErb WNBkOb ']/div[@class='ZKCDEc']/div[@class='RZ66Rb FgCUCc']/button[@class='aoRNLd kn2E5e NMjTrf lvtCsd ']/img").get_attribute('src')

        comentario = 'Dirección encontrada'
    
    except:

        try:
            wait(driver, 4).until(ec.element_to_be_clickable((By.CLASS_NAME, "hfpxzc")))
            boton = driver.find_element(By.CLASS_NAME, "hfpxzc")
            boton.click()

            time.sleep(3)

            coordenadas = driver.find_element(By.CSS_SELECTOR, 'meta[itemprop=image]').get_attribute('content')

            latitud = float(coordenadas[coordenadas.find('-'):coordenadas.find("%")])
            longitud =float(coordenadas[coordenadas.find('-', coordenadas.find("-")+1):coordenadas.find("&")])

            wait(driver, 4).until(ec.visibility_of_element_located((By.XPATH, "html/body/div[@id='app-container']/div[@id='content-container']/div[@id='QA0Szd']/div/div[@class='XltNde tTVLSc']/div[@class='w6VYqd']/div[@class='bJzME Hu9e2e tTVLSc']/div[@class='k7jAl lJ3Kh miFGmb']/div[@tabindex='-1']/div[@class='aIFcqe']/div[@class='m6QErb WNBkOb ']/div[@class='m6QErb DxyBCb kA9KIf dS8AEf ']/div[@class='ZKCDEc']/div[@class='RZ66Rb FgCUCc']/button[@class='aoRNLd kn2E5e NMjTrf lvtCsd ']/img")))
            imagen = driver.find_element(By.XPATH, "html/body/div[@id='app-container']/div[@id='content-container']/div[@id='QA0Szd']/div/div[@class='XltNde tTVLSc']/div[@class='w6VYqd']/div[@class='bJzME Hu9e2e tTVLSc']/div[@class='k7jAl lJ3Kh miFGmb']/div[@tabindex='-1']/div[@class='aIFcqe']/div[@class='m6QErb WNBkOb ']/div[@class='m6QErb DxyBCb kA9KIf dS8AEf ']/div[@class='ZKCDEc']/div[@class='RZ66Rb FgCUCc']/button[@class='aoRNLd kn2E5e NMjTrf lvtCsd ']/img").get_attribute('src')

            comentario = 'Más de una dirección encontrada, toma la primera'
            
            print(comentario)

        except:
            
            comentario = "Dirección no encontrada"
            print(comentario)
        
            latitud = np.nan
            longitud = np.nan

            imagen = ''

    driver.close()

    return(latitud, longitud)

In [15]:
coord = obtener_coordenadas(dir)
coord

(-33.4504715, -70.5974685)

In [16]:
df_tiendas_tottus = pd.read_excel("web-scraper-tottus/df_tiendas_tottus.xlsx")
df_tiendas_tottus['empresa'] = 'Tottus'
df_tiendas_tottus.head(3)

,region,comuna,direccion,imagen,latitud,longitud,comentario,empresa
0,Región Metropolitana,Alameda,"Av Libertador Bernardo O'Higgins 3470, Santiag...",https://lh5.googleusercontent.com/p/AF1QipNBTs...,-33.452419,-70.682373,Dirección encontrada,Tottus
1,Región Metropolitana,Buín,"San Martín 174, Buin\nHorario\nLunes a Domingo...",https://lh5.googleusercontent.com/p/AF1QipObZ2...,-33.731760,-70.734828,Dirección encontrada,Tottus
2,Región Metropolitana,Catedral,"Catedral 1850, Santiago\nHorario\nLunes a Vier...",https://lh5.googleusercontent.com/p/AF1QipNgsl...,-33.438856,-70.663054,Dirección encontrada,Tottus


In [17]:
lista_tottus = df_tiendas_tottus[['empresa', 'region', 'comuna', 'direccion', 'latitud', 'longitud']].values.tolist()

In [18]:
df_tiendas_lider = pd.read_excel("web-scraper-lider/df_tiendas_lider.xlsx")
df_tiendas_lider['empresa'] = 'Lider'
df_tiendas_lider.head(3)

,supermercado,comuna,direccion,disponibilidad,region,hora_apertura,hora_cierre,imagen,latitud,longitud,comentario,empresa
0,Express de Lider,Renca,"San Diego 219, Santiago",Abierto,Metropolitana,08:00,21:30,https://lh5.googleusercontent.com/p/AF1QipNcMr...,-33.447774,-70.651169,Dirección encontrada,Lider
1,Express de Lider,Santiago,"Curicó 517, Santiago",Abierto,Metropolitana,08:00,21:30,https://lh5.googleusercontent.com/p/AF1QipOznS...,-33.446068,-70.643188,Dirección encontrada,Lider
2,Express de Lider,Santiago,"San Pablo 2902, Santiago",Abierto,Metropolitana,08:00,21:30,https://lh5.googleusercontent.com/p/AF1QipNoxJ...,-33.435443,-70.673451,"Más de una dirección encontrada, toma la primera",Lider


In [19]:
lista_lider = df_tiendas_lider[['empresa', 'region', 'comuna', 'direccion' ,'latitud', 'longitud']].values.tolist()

In [20]:
df_tiendas_jumbo = pd.read_excel("web-scraper-jumbo/df_tiendas_jumbo.xlsx")
df_tiendas_jumbo['empresa'] = 'Jumbo'
df_tiendas_jumbo.head(3)

,local,direccion,url_maps,lat,lon,imagen,latitud,longitud,comentario,comuna,region,empresa
0,Jumbo Pie Andino,"Avenida paseo pie andino 4770 , Lo Barnechea, ...",https://www.google.com/maps/search/?api=1&quer...,-33.325961,-70.517112,https://lh5.googleusercontent.com/p/AF1QipN7Jg...,-33.325562,-70.517186,Dirección encontrada,Lo Barnechea,Región Metropolitana,Jumbo
1,Jumbo Chamisero,"Av. Santa María 14.601, Colina, Región Metropo...",https://www.google.com/maps/search/?api=1&quer...,-33.321480,-70.645935,https://lh5.googleusercontent.com/p/AF1QipM-vf...,-33.321492,-70.646300,Dirección encontrada,Colina,Región Metropolitana,Jumbo
2,Jumbo Puertas de Chicureo,"Avendida Chicureo 130, Colina, Región Metropol...",https://www.google.com/maps/search/?api=1&quer...,-33.285435,-70.700211,NaN,NaN,NaN,Dirección no encontrada,Colina,Región Metropolitana,Jumbo


In [21]:
lista_jumbo = df_tiendas_jumbo[['empresa', 'region', 'comuna', 'direccion', 'latitud', 'longitud']].values.tolist()

In [22]:
lista = []
lista.extend(lista_tottus)
lista.extend(lista_lider)
lista.extend(lista_jumbo)

In [23]:
len(lista), len(lista_tottus), len(lista_lider), len(lista_jumbo)

(473, 72, 380, 21)

In [24]:
results = []

for e, r, c, d, lat, lon in lista:
    distancia = haversine(coord, (lat, lon))
    results.append([e, r, c, d, lat, lon, distancia])

In [25]:
df_results = pd.DataFrame(results, columns=['empresa', 'region', 'comuna', 'direccion', 'latitud', 'longitud', 'distancia'])

In [26]:
df_results.sort_values('distancia').head(10)

,empresa,region,comuna,direccion,latitud,longitud,distancia
91,Lider,Metropolitana,Ñuñoa,"Irarrázaval 710, Ñuñoa",-33.453682,-70.598652,0.373521
87,Lider,Metropolitana,Ñuñoa,"General Jose Artigas 2877, Ñuñoa",-33.445169,-70.599452,0.617690
143,Lider,Metropolitana,Ñuñoa,Av. Irarrázaval 5353,-33.453682,-70.590598,0.730558
146,Lider,Metropolitana,Ñuñoa,Irarrázaval Av. 2410,-33.459082,-70.600859,1.007731
319,Lider,Metropolitana,Providencia,"Pedro de Valdivia 1885, Providencia",-33.440042,-70.608676,1.557674
95,Lider,Metropolitana,Providencia,"Los Leones 1875, Providencia",-33.436691,-70.601307,1.573137
82,Lider,Metropolitana,Ñuñoa,"Coventry 601, Ñuñoa",-33.448633,-70.580426,1.594289
465,Jumbo,Región Metropolitana,Ñuñoa,"Av. José Pedro Alessandri 1166, Ñuñoa, Región ...",-33.465232,-70.597481,1.641307
148,Lider,Metropolitana,Ñuñoa,Jose Domingo Cañas 1580,-33.455950,-70.615054,1.741466
75,Lider,Metropolitana,Providencia,"Providencia 1630, Providencia",-33.434791,-70.611593,2.181152
